In [5]:
import pandas as pd

In [6]:
import json
with open("results/evaluation/bart-test-extrinsic-100-summaries.json", "r") as f:
    json_sums = json.load(f)
df = pd.DataFrame(json_sums.values(), index=list(json_sums.keys()))
df.head()

,baseline-bart_count_extrinsic,baseline-bart_count_extrinsic_factual,baseline-bart_extrinsic_factuality_ratio,baseline-bart_has_extrinsic_and_fully_factual,baseline-bart_has_failed,baseline-bart_has_unknown,baseline-bart_is_factual,baseline-bart_is_non_factual,baseline-bart_is_non_factual_extrinsic,baseline-bart_is_non_factual_intrinsic,...,pinocchio_extrinsic_factuality_ratio,pinocchio_has_extrinsic_and_fully_factual,pinocchio_has_failed,pinocchio_has_unknown,pinocchio_is_factual,pinocchio_is_non_factual,pinocchio_is_non_factual_extrinsic,pinocchio_is_non_factual_intrinsic,pinocchio_is_skipped,pinocchio_summary
14702705,1,1,1.0,1,False,False,True,False,False,False,...,1.0,1,False,False,True,False,False,False,False,The Kingdom of Saudi Arabia is the largest cou...
14766406,1,0,0.0,0,False,False,False,True,True,False,...,0.0,0,False,False,False,True,True,False,False,An 18-year-old man has been arrested in connec...
17996567,1,0,0.0,0,False,False,False,True,True,False,...,0.0,0,False,False,False,True,True,False,False,Plans to cut home-to-school transport subsidie...
18239960,1,0,0.0,0,False,False,False,True,True,False,...,0.0,0,False,False,False,True,True,False,False,"There is a ""pattern of behaviour"" within the p..."
19541955,1,1,1.0,1,False,False,True,False,False,False,...,1.0,1,False,False,True,False,False,False,False,India may have won just one medal at the Londo...


In [7]:
def get_baseline_diff(df, model, metric):
    return pd.crosstab(index=df[f"baseline-bart_{metric}"], columns=df[f"{model}_{metric}"])

df_gef = get_baseline_diff(df, "fbs_classifier", "is_factual")

from scipy.stats import fisher_exact, barnard_exact
from scipy.stats import chi2_contingency

print("fisher", fisher_exact(df_gef)[1])
print("barnard", barnard_exact(df_gef).pvalue)
print("chi-sq", chi2_contingency(df_gef)[1])

df_gef

fisher 9.670592759133418e-15
barnard 1.3564572824547934e-14
chi-sq 7.955196371763444e-13


fbs_classifier_is_factual,False,True
baseline-bart_is_factual,,
False,46,12
True,2,40


In [8]:
from statsmodels.stats.contingency_tables import mcnemar

for model in ["corrector", "pinocchio", "meng-rl", "fbs_classifier", "fbs_oracle"]:

    for metric in ["is_factual", "has_extrinsic_and_fully_factual"]:
        diff_metric = get_baseline_diff(df, model, metric)
        baseline_mean = df[f"baseline-bart_{metric}"].mean()
        model_mean = df[f"{model}_{metric}"].mean()
        print(f"{model} {metric}: {model_mean} (baseline diff: {model_mean - baseline_mean})")
        print(f"{model} {metric} baseline diff p-value: {mcnemar(diff_metric).pvalue}")

    print()

corrector is_factual: 0.42 (baseline diff: 0.0)
corrector is_factual baseline diff p-value: 1.0
corrector has_extrinsic_and_fully_factual: 0.45 (baseline diff: 0.010000000000000009)
corrector has_extrinsic_and_fully_factual baseline diff p-value: 1.0

pinocchio is_factual: 0.43 (baseline diff: 0.010000000000000009)
pinocchio is_factual baseline diff p-value: 1.0
pinocchio has_extrinsic_and_fully_factual: 0.43 (baseline diff: -0.010000000000000009)
pinocchio has_extrinsic_and_fully_factual baseline diff p-value: 1.0

meng-rl is_factual: 0.55 (baseline diff: 0.13000000000000006)
meng-rl is_factual baseline diff p-value: 0.004425048828125
meng-rl has_extrinsic_and_fully_factual: 0.39 (baseline diff: -0.04999999999999999)
meng-rl has_extrinsic_and_fully_factual baseline diff p-value: 0.359283447265625

fbs_classifier is_factual: 0.52 (baseline diff: 0.10000000000000003)
fbs_classifier is_factual baseline diff p-value: 0.012939453125
fbs_classifier has_extrinsic_and_fully_factual: 0.47 (bas